In [1]:
import os

In [2]:
%pwd

'c:\\ML_Projects\\MLflow_Implementation\\MLFlow-End-to-End-implementation\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\ML_Projects\\MLflow_Implementation\\MLFlow-End-to-End-implementation'

In [5]:
import dagshub
dagshub.init(repo_owner='vipinvsist', repo_name='MLFlow-End-to-End-implementation', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as vipinvsist

Initialized MLflow to track repo "vipinvsist/MLFlow-End-to-End-implementation"

Repository vipinvsist/MLFlow-End-to-End-implementation initialized!

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModeEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlflowProject.constants import *
from mlflowProject.utils.common import read_yaml,create_directories,save_json


class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModeEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.Lasso
        schema = self.schema.TARGET_COLUMN
        print("-----------------",params)

        create_directories([config.root_dir])
        target_column = list(schema.keys())[0] if schema else "OutletSales"

        model_evaluation_config = ModeEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path = config.test_data_path,
            model_path= config.model_path,
            all_params=params,
            metric_file_name= config.metric_file_name,
            target_column= target_column,
            mlflow_uri= "https://dagshub.com/vipinvsist/MLFlow-End-to-End-implementation.mlflow"

            )
        return model_evaluation_config

In [8]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [9]:
class ModeEvaluation:
    def __init__(self,config :ModeEvaluationConfig):
        self.config = config

    def eval_metrices(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)

        return rmse, mae, r2
      
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        test_x=pd.get_dummies(test_x)
        test_y=np.log(test_y)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse,mae,r2) = self.eval_metrices(test_y,predicted_qualities)

            # saving the loaded  metrices

            scores = {"rmse": rmse, "mae" : mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name),data =scores)

            for key, value in self.config.all_params.items():
                mlflow.log_param(key, value)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("r2",r2)
            mlflow.log_metric("mae",mae)


            if tracking_url_type_store !='file' :
                # Register the model
                # There are other ways also , follow below link tknow more:
                # https://mlflow.org/docs/latest/model-registery.html#api-workflow
                model_uri = mlflow.sklearn.log_model(model, "model")  
            else:
                mlflow.sklearn.load_model(model, "model")

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModeEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-11-23 00:30:06,656: INFO: common: json file loaded sucessfully from: C:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation\config\config.yaml]
[2024-11-23 00:30:06,660: INFO: common: json file loaded sucessfully from: C:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation\params.yaml]
[2024-11-23 00:30:06,667: INFO: common: json file loaded sucessfully from: C:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation\schema.yaml]
[2024-11-23 00:30:06,670: INFO: common: created directory at artifacts]
----------------- {'alpha': 0.001}
[2024-11-23 00:30:06,672: INFO: common: created directory at artifacts/model_evaluation]
[2024-11-23 00:30:07,485: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\ML_Projects\MLflow_Implementation\mlflow_implementation_project_env\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
